# Mandelbrot Set

This notebook provides an interactive look into the Mandelbrot Set. Here we provide a basic implementation to compute the set given bounds, number of steps, and a max iterations.

## Defining Functions

Below we implement a few functions for computing the Mandelbrot Set. The first set of functions below is a straight forward implementation; we provide it to illustrate how it all works.

In [ ]:
def mandelbrot(c, max_iterations):
    z = 0.0j
    iterations = 0
    while abs(z) < 2.0 and iterations < max_iterations:
        z = z**2 + c
        iterations += 1
    return iterations

def compute(x_min, x_max, y_min, y_max, steps, max_iterations):
    step = (x_max - x_min) / steps
    data = [[None]*steps for _y in range(steps)]
    for yidx in range(steps):
        for xidx in range(steps):
            data[yidx][xidx] = mandelbrot(complex(x_min + xidx * step, y_min + yidx * step), max_iterations)
    return data

Here we also provide a vectorized implementation for improved performance. We use a tool called `numba` that takes our functions and optimizes them so that they can be applied to entire arrays of data. We then rewrite our compute function to work with our special vectorized function.

In [ ]:
import numba as nb
import numpy as np

vmandelbrot = nb.vectorize(mandelbrot)

def vcompute(x_min, x_max, y_min, y_max, steps, max_iterations):
    step = (x_max - x_min) / steps
    dx, dy = np.indices((steps, steps))
    dx = x_min + step * dx
    dy = y_min + step * dy
    
    data = np.array(np.vectorize(complex)(dx.T, dy.T))
    
    return vmandelbrot(data, max_iterations)

## Testing Our Functions

The Mandelbrot Set is roughly bounded by the square with vertices (-2.0, -1.25), (0.5, -1.25), (0.5, 1.25), (-2.0, 1.25), so we will use this to generate a high level view. First let's set up a few variables and compute our first Mandelbrot Set.

In [ ]:
x_min = -2.0
x_max = 0.5
y_min = -1.25
y_max = 1.25
steps = 100
max_iterations = 100

ms = vcompute(x_min, x_max, y_min, y_max, steps, max_iterations)
ms

Well thats not very exciting? We need to actually visualize it! Let's take a step back and reduce the number of steps from 100 to 10 (just temporarily).

In [ ]:
ms = vcompute(x_min, x_max, y_min, y_max, 10, max_iterations)
ms

### First Image

What we actually have here is image-data! Each element represents a pixel, and each value of each element represents a different color. But 1, 2, 3, etc. are clearly not colors. What we have is something that is actually a heatmap! Let's recompute with the larger number of steps and use bokeh to produce the heatmap as an image.

In [ ]:
from bokeh.io import output_notebook, show         # import tools for showing in a Jupyter notebook
from bokeh.models import ColumnDataSource, Range1d # import structures for data sources and ranges
from bokeh.plotting import figure                  # import bokeh figures

output_notebook()                                  # tell bokeh to prepare itself for use within a Jupyter notebook

# recompute the set
ms = vcompute(x_min, x_max, y_min, y_max, steps, max_iterations)

fig = figure()                   # create a bokeh figure
source = ColumnDataSource(data={ # create a column data source to bridge the data and the figure
    'image': [ms],               # set the pixel data
    'x': [x_min],                # set the lower left corner anchor
    'y': [y_min],                # set the lower left corner anchor
    'dw': [x_max-x_min],         # set the width-dimension
    'dh': [y_max-y_min],         # set the height-dimension
})

fig.image(image='image', x='x', y='y', dw='dw', dh='dh', palette='Spectral11', source=source) # generate the image
fig.x_range = Range1d(x_min, x_max)                                                           # fix the x-axis
fig.y_range = Range1d(y_min, y_max)                                                           # fix the y-axis

show(fig)

We can up the number of steps to get more detail, which should improve the experience of zooming.

In [ ]:
steps = 1000
ms = vcompute(x_min, x_max, y_min, y_max, steps, max_iterations)
ms

source.data = {
    'image': [ms],
    'x': [x_min],
    'y': [y_min],
    'dw': [x_max-x_min],
    'dh': [y_max-y_min],
}
show(fig)

## Initial Dashboard

As we increase the number of steps we take we are ultimately increasing the resolution of our image. This is why we can zoom in a bit more and see more detail. It is impractical however to continually render larger resolutions so that we can dive deeper. We want to configure our plot to recompute itself as we zoom in. That way we can work with a fixed resolution, but change the region over which we operate. This will give us an effect of being able to zoom infinitely.

This is not easy to set up though. What we are ultimately trying to do is trigger a computation anytime the user zooms or pans. We also want to provide controls for the numbers of steps (resolution) and the maximum iterations (which controls the color spread).

In [ ]:
import panel as pn
pn.extension()

steps_slider = pn.widgets.IntSlider(name='Steps', start=1, end=1000, step=1, value=50)              # control steps (image resolution)
max_iterations_radio = pn.widgets.RadioButtonGroup(name='Max Iterations', options=[100, 500, 1000]) # control max iterations (color spread)

# create a few disabled text widgets for displaying data
# by disabling them we can prorammatically update them and
# prevent the user from updating them.
x_min_text = pn.widgets.TextInput(name='x-min', value='-2.0', disabled=True)
x_max_text = pn.widgets.TextInput(name='x-max', value='0.5', disabled=True)
y_min_text = pn.widgets.TextInput(name='y-min', value='-1.25', disabled=True)
y_max_text = pn.widgets.TextInput(name='y-max', value='1.25', disabled=True)

# create an empty Column Data Source (CDS)
source = ColumnDataSource(data={
    'image': [],
    'x': [],
    'y': [],
    'dw': [],
    'dh': [],
})


# create the figure and the image, mapping from the CDS
fig = figure()
fig.image(image='image', x='x', y='y', dw='dw', dh='dh', palette='Spectral11', source=source)

# write a function that recomputes the set and update the CDS
def visualize_mandelbrot(steps, max_iterations):
    x_min = float(x_min_text.value)
    x_max = float(x_max_text.value)
    y_min = float(y_min_text.value)
    y_max = float(y_max_text.value)

    # compute the new set
    ms = vcompute(x_min, x_max, y_min, y_max, steps, max_iterations)
    
    # update the CDS
    source.data = {
        'image': [ms],
        'x': [x_min],
        'y': [y_min],
        'dw': [x_max-x_min],
        'dh': [y_max-y_min],
    }
    
    # ensure the ranges are set properly
    fig.x_range=Range1d(x_min, x_max)
    fig.y_range=Range1d(y_min, y_max)
    
    # return the existing figure, so that panel knows what to update with
    return fig

interaction = pn.interact(visualize_mandelbrot, steps=steps_slider, max_iterations=max_iterations_radio)
interaction

## Micro-Dashboard

Below we expand upon the autogenerated `panel` layout.

In [ ]:
from bokeh import events

steps_slider = pn.widgets.IntSlider(name='Steps', start=1, end=1000, step=1, value=50)
max_iterations_radio = pn.widgets.RadioButtonGroup(name='Max Iterations', options=[100, 500, 1000])
reset = pn.widgets.Button(name='Reset', )

x_min_text = pn.widgets.TextInput(name='x-min', value='-2.0', disabled=True)
x_max_text = pn.widgets.TextInput(name='x-max', value='0.5', disabled=True)
y_min_text = pn.widgets.TextInput(name='y-min', value='-1.25', disabled=True)
y_max_text = pn.widgets.TextInput(name='y-max', value='1.25', disabled=True)


def visualize_mandelbrot(steps, max_iterations):
    x_min = float(x_min_text.value)
    x_max = float(x_max_text.value)
    y_min = float(y_min_text.value)
    y_max = float(y_max_text.value)
    
    ms = vcompute(x_min, x_max, y_min, y_max, steps, max_iterations)
    
    source = ColumnDataSource(data={
        'image': [ms],
        'x': [x_min],
        'y': [y_min],
        'dw': [x_max-x_min],
        'dh': [y_max-y_min],
    })
    fig = figure(active_scroll='wheel_zoom')
    fig.image(image='image', x='x', y='y', dw='dw', dh='dh', palette='Spectral11', source=source)

    fig.x_range=Range1d(x_min, x_max)
    fig.y_range=Range1d(y_min, y_max)
    
    fig.x_range.reset_start = -2.0
    fig.x_range.reset_end = 0.5
    fig.y_range.reset_start = -1.25
    fig.y_range.reset_end = 1.25
    
    def lod_callback(*args):
        x_min_text.value = str(fig.x_range.start)
        x_max_text.value = str(fig.x_range.end)
        y_min_text.value = str(fig.y_range.start)
        y_max_text.value = str(fig.y_range.end)
        lod_ms = vcompute(fig.x_range.start, fig.x_range.end, fig.y_range.start, fig.y_range.end, steps_slider.value, max_iterations_radio.value)
        source.data = {
            'image': [lod_ms],
            'x': [fig.x_range.start],
            'y': [fig.y_range.start],
            'dw': [fig.x_range.end-fig.x_range.start],
            'dh': [fig.y_range.end-fig.y_range.start],
        }

    fig.on_event(events.LODEnd, lod_callback)
    fig.on_event(events.Reset, lod_callback)

    return fig

interaction = pn.interact(visualize_mandelbrot, steps=steps_slider, max_iterations=max_iterations_radio, throttled=True)

header = '''# Basic Mandelbrot Set Explorer

This is a basic Mandelbrot Set explorer. We can dive deep into the set and control the resolution of our image

'''

layout = pn.Column(
    pn.pane.Markdown(header),
    pn.Row(
        pn.Column(x_min_text,x_max_text,y_min_text,y_max_text, *interaction[0]),
        interaction[1][0]
    )
)
layout